In [1]:
#!pip install rdkit
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
#!pip install torch_geometric

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, ParameterGrid
from sklearn.metrics import mean_absolute_error, mean_squared_error

from scipy.stats import pearsonr

In [ ]:
df = pd.read_csv("C:\\itmo\\BONUS TRACK\\final_project\\data\\NN_ML.csv", delimiter=",")
df

In [ ]:
"""Загрузка данных"""
# Определяю типы атомов в ".xyz"-файлах
# Расстояния между ними будут использованы пакетом RDKit для присвоения типов связей
# P.S. ".xyz"-файлы не содержат "connectivity", поэтому визуализаторы по типу "ChemCraft" и "Avogadro" используют такой подход для присвоения связей

def read_xyz_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:] 
        elements = []
        for line in lines:
            parts = line.split() 
            element = parts[0]  
            elements.append(element)  
    return elements 

def identify_atom_types(directory_path):
    atom_types = set()
    for filename in os.listdir(directory_path):
        if filename.endswith('.xyz'):
            file_path = os.path.join(directory_path, filename)
            elements = read_xyz_file(file_path)
            atom_types.update(elements)
    return atom_types

directory_path = 'C:\\itmo\\BONUS TRACK\\final_project\\xyz'
atom_types = identify_atom_types(directory_path)
print("Atom types found in .xyz files:")
for atom in sorted(atom_types):
    print(atom)

In [ ]:
"""Загрузка данных"""
# Функция read_xyz_file() читает ".xyz"--файлы (типы атомов + координаты)
# Функция check_valence() проверяет валентность атомов
# Функция bond_exists() проверяет наличие связи
# Функция xyz_to_rdkit_mol() конвертирует ".xyz"-данные в RDKit-молекулы (типы связей присвоены в соответствии с расстоянием между атомами)
# Цикл выдаёт "SMILES" RDKit-молекул (при необходимости, можно сравнить связи и структуры молекул с оригинальными ".xyz"-файлами)

def read_xyz_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:] 
        coordinates = []
        elements = []
        for line in lines:
            parts = line.split() 
            element = parts[0]  
            x, y, z = float(parts[1]), float(parts[2]), float(parts[3]) 
            elements.append(element) 
            coordinates.append([x, y, z]) 
    return elements, coordinates

def check_valence(mol, atom_idx):
    try:
        Chem.SanitizeMol(mol)
        return True
    except Chem.AtomValenceException:
        return False

def bond_exists(mol, atom_idx1, atom_idx2):
    bond = mol.GetBondBetweenAtoms(atom_idx1, atom_idx2)
    return bond is not None

def xyz_to_rdkit_mol(elements, coordinates):
    single_bonds = {
        ('C', 'H'): 1.09,
        ('C', 'C'): 1.54,
        ('C', 'N'): 1.47,
        ('C', 'O'): 1.43,
        ('H', 'H'): 0.74,
        ('N', 'H'): 1.01,
        ('O', 'H'): 0.96,
        ('N', 'N'): 1.45,
        ('N', 'O'): 1.40,
        ('O', 'O'): 1.48,
    }

    double_bonds = {
        ('C', 'C'): 1.30,
        ('C', 'N'): 1.30,
        ('C', 'O'): 1.23,
        ('N', 'O'): 1.22,
    }

    triple_bonds = {
        ('C', 'C'): 1.20,
        ('C', 'N'): 1.16,
    }

    mol = Chem.RWMol()
    conf = Chem.Conformer(len(elements))

    for i, element in enumerate(elements):
        atom = Chem.Atom(element)
        mol.AddAtom(atom)
        conf.SetAtomPosition(i, Chem.rdGeometry.Point3D(*coordinates[i]))

    mol.AddConformer(conf)

    num_atoms = len(elements)
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            distance = np.linalg.norm(np.array(coordinates[i]) - np.array(coordinates[j]))
            pair = (elements[i], elements[j])
            if pair not in single_bonds:
                pair = (elements[j], elements[i])

            if bond_exists(mol, i, j):
                continue

            if pair in triple_bonds and distance < triple_bonds[pair] * 1.04:
                if not bond_exists(mol, i, j):
                    mol.AddBond(i, j, Chem.BondType.TRIPLE)
                    if not check_valence(mol, i) or not check_valence(mol, j):
                        mol.RemoveBond(i, j)
                        continue

            if pair in double_bonds and distance < double_bonds[pair] * 1.04:
                if not bond_exists(mol, i, j):
                    mol.AddBond(i, j, Chem.BondType.DOUBLE)
                    if not check_valence(mol, i) or not check_valence(mol, j):
                        mol.RemoveBond(i, j)
                        continue

            if pair in single_bonds and distance < single_bonds[pair] * 1.04:
                if not bond_exists(mol, i, j): 
                    mol.AddBond(i, j, Chem.BondType.SINGLE)
                    if not check_valence(mol, i) or not check_valence(mol, j):
                        mol.RemoveBond(i, j)

    try:
        Chem.SanitizeMol(mol)
        mol = Chem.AddHs(mol)
        Chem.SanitizeMol(mol)
    except Chem.AtomValenceException as e:
        print(f"Valence error for molecule: {e}")
        return None
    except Chem.AtomSanitizeException as e:
        print(f"Sanitization error for molecule: {e}")
        return None

    return mol

"""directory_path = 'C:\\itmo\\BONUS TRACK\\final_project\\xyz'
for filename in os.listdir(directory_path):
    if filename.endswith('.xyz'):
        file_path = os.path.join(directory_path, filename)
        elements, coordinates = read_xyz_file(file_path)
        mol = xyz_to_rdkit_mol(elements, coordinates)
        if mol:
            print(f"{filename}: {Chem.MolToSmiles(mol)}")
        else:
            print(f"Failed to create RDKit molecule for {filename}.")"""

In [ ]:
"""Загрузка данных"""
# Функция прводит вычисления RDKit-признаков для каждой молекулы

def compute_rdkit_features(mol):
    if mol is None:
        return None
    morgan_fp = Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    return np.array(morgan_fp)

In [ ]:
"""Загрузка данных"""
# Функция для графового представления БД
# HF-3с столбцы & RdKit-столбец - признаки нодов
# Порог в 3.5 А установлен для эджей (они нужны для выявления геометрических связей между нодами [атомами], а не химических)
# Добавил DFT энергии (таргеты) в качестве меток

def create_graph_data(df):
    graph_data_list = []
    
    for idx, row in df.iterrows():
        coordinates = np.array(row['coordinates'])
        num_atoms = coordinates.shape[0]

        hf_features = row[['hf_gibbs_free_energy_ev', 'hf_electronic_energy_ev', 'hf_entropy_ev',
                           'hf_enthalpy_ev', 'hf_dipole_moment_d', 'hf_homo_ev', 'hf_lumo_ev', 'mass_au']].astype(float).values

        rdkit_features = row['rdkit_features']
        if rdkit_features is not None:
            hf_features = np.concatenate([hf_features, rdkit_features])

        node_features = np.tile(hf_features, (num_atoms, 1))
        node_features = torch.tensor(node_features, dtype=torch.float)

        edge_index = []
        for i in range(num_atoms):
            for j in range(i + 1, num_atoms):
                distance = np.linalg.norm(coordinates[i] - coordinates[j])
                if distance < 3.5:
                    edge_index.append([i, j])
                    edge_index.append([j, i])
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        
        graph_data = Data(x=node_features, edge_index=edge_index)
        graph_data.y = torch.tensor(row[['dft_gibbs_free_energy_ev', 'dft_electronic_energy_ev', 'dft_entropy_ev',
                                         'dft_enthalpy_ev', 'dft_dipole_moment_d', 'dft_homo_ev', 'dft_lumo_ev']].astype(float).values, dtype=torch.float)
        graph_data_list.append(graph_data)
    
    return graph_data_list

In [ ]:
"""Загрузка данных"""
# Добавил столбцы с координатами и атомами

xyz_folder = "C:/itmo/BONUS TRACK/final_project/xyz"

df['coordinates'] = [[] for _ in range(len(df))]
df['elements'] = [[] for _ in range(len(df))]

for idx, row in df.iterrows():
    file_name = f"{row['name']}.xyz"
    file_path = os.path.join(xyz_folder, file_name)
    if os.path.exists(file_path):
        elements, coordinates = read_xyz_file(file_path)
        df.at[idx, 'coordinates'] = coordinates
        df.at[idx, 'elements'] = elements

In [ ]:
"""Загрузка данных"""
# Добавил столбец с RDKit-признаками

df['rdkit_features'] = df.apply(lambda row: compute_rdkit_features(xyz_to_rdkit_mol(row['elements'], row['coordinates'])), axis=1)

In [ ]:
"""Загрузка данных"""
# Список графов

graph_data_list = create_graph_data(df)
graph_data_list

In [ ]:
"""Загрузка данных"""
# У первого графа 34 нод, а количество крпизнаков у ноды = 2056 (2048 фингерпринтов + 7 HF-3c столбцов + mass_au)
# Каждый эдж представлен двумя нодами, а общее количество эджей в графе = 310 (при пороге в 3.5 А)
# 7 - количество таргетов

graph_data_list[0]

In [ ]:
"""Загрузка данных"""
# Графы были сгенерированы для каждого наблюдения 
# Проверяю количество графов 

num_graphs = len(graph_data_list)
print(f"Number of graphs: {num_graphs}")

In [ ]:
"""Разведочный анализ данных (EDA)"""
# Визуализирую распределение количества нодов и эджей для каждого графа

num_nodes = []
num_edges = []

for graph_data in graph_data_list:
    num_nodes.append(graph_data.num_nodes)
    num_edges.append(graph_data.num_edges // 2) 

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.hist(num_nodes, bins=20, edgecolor='black')
plt.xlabel('Number of Nodes', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Number of Nodes', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=10)

plt.subplot(1, 2, 2)
plt.hist(num_edges, bins=20, edgecolor='black')
plt.xlabel('Number of Edges', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Number of Edges', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=10)

plt.tight_layout()
plt.show()

In [ ]:
"""Разведочный анализ данных (EDA)"""
# Визуализирую первые 5 графов

num_graphs_to_visualize = 5

for i in range(num_graphs_to_visualize):
    graph_data = graph_data_list[i]
    num_nodes = graph_data.num_nodes
    edge_index = graph_data.edge_index.numpy().T
    
    G = nx.Graph()
    G.add_nodes_from(range(num_nodes))
    G.add_edges_from(edge_index)
    
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G) 
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, edge_color='gray', linewidths=1, font_size=12)
    plt.title(f'Graph {i+1}')
    plt.show()

In [ ]:
"""Разведочный анализ данных (EDA)"""
# Матрица корреляции признаков нод (HF-3c признаков + mass_au) и таргетов (DFT)

hf_features = ['hf_gibbs_free_energy_ev', 'hf_electronic_energy_ev', 'hf_entropy_ev',
               'hf_enthalpy_ev', 'hf_dipole_moment_d', 'hf_homo_ev', 'hf_lumo_ev']
dft_features = ['dft_gibbs_free_energy_ev', 'dft_electronic_energy_ev', 'dft_entropy_ev',
                'dft_enthalpy_ev', 'dft_dipole_moment_d', 'dft_homo_ev', 'dft_lumo_ev']

correlation_matrix = df[hf_features + dft_features].corr().loc[hf_features, dft_features]

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title('Correlation Matrix between HF Features and DFT Targets')
plt.show()

In [ ]:
"""Разведочный анализ данных (EDA)"""
# Cредняя абсолютная корреляция каждого HF-3c признака с любым таргетом DFT
# Максимальная абсолютная корреляция каждого HF-3c признака с любым таргетом DFT

average_correlations = correlation_matrix.abs().mean(axis=1)
ranked_features_avg = average_correlations.sort_values(ascending=False)
print("\nHF features ranked by average absolute correlation:\n",ranked_features_avg)

max_correlations = correlation_matrix.abs().max(axis=1)
ranked_features_max = max_correlations.sort_values(ascending=False)
print("\nHF features ranked by maximum absolute correlation:\n",ranked_features_max)

In [ ]:
"""Разведочный анализ данных (EDA)"""
# Функция для гистограмм распределения значений

def plot_histograms(features, title):
    n_cols = 3
    n_rows = (len(features) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 12))
    fig.suptitle(title, fontsize=20)

    for i, feature in enumerate(features):
        ax = axes[i // n_cols, i % n_cols]
        ax.hist(df[feature], bins=30, edgecolor='k', alpha=0.7)
        ax.set_title(feature)
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')

    for i in range(len(features), n_rows * n_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
"""Разведочный анализ данных (EDA)"""
# HF-3c (гистограммы распределения)

plot_histograms(hf_features, 'Distribution Histograms for HF Features')

In [ ]:
"""Разведочный анализ данных (EDA)"""
# DFT (гистограммы распределения)

plot_histograms(dft_features, 'Distribution Histograms for DFT Targets')

In [ ]:
"""Разведочный анализ данных (EDA)"""
# Функция для диаграмм "Ящик с усами"

def plot_boxplots(features, title):
    n_cols = 3
    n_rows = (len(features) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 12))
    fig.suptitle(title, fontsize=20)

    for i, feature in enumerate(features):
        ax = axes[i // n_cols, i % n_cols]
        ax.boxplot(df[feature].dropna(), vert=True, patch_artist=True)
        ax.set_title(feature)
        ax.set_ylabel('Value')

    for i in range(len(features), n_rows * n_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
"""Разведочный анализ данных (EDA)"""
# HF-3c ("Ящик с усами")

plot_boxplots(hf_features, 'Box-and-Whisker Diagrams for HF Features')

In [ ]:
"""Разведочный анализ данных (EDA)"""
# DFT ("Ящик с усами")

plot_boxplots(dft_features, 'Box-and-Whisker Diagrams for DFT Targets')

In [ ]:
"""Разведочный анализ данных (EDA)"""
# P-значения корреляции HF-3c признаков с каждым DFT таргетом

p_values = pd.DataFrame(index=hf_features, columns=dft_features)

for hf in hf_features:
    for dft in dft_features:
        _, p_value = pearsonr(df[hf], df[dft])
        p_values.loc[hf, dft] = p_value

print("\nP-values for correlations between HF features and DFT targets:\n", p_values)

In [ ]:
"""Графовая сверточная нейросеть"""
# Делю на тренировочную, тестовую и валидационную выборки 
# 80 / 10 / 10

train_val_graphs, test_graphs = train_test_split(graph_data_list, test_size=0.1, random_state=42)
train_graphs, val_graphs = train_test_split(train_val_graphs, test_size=0.1, random_state=42)

print(f"Number of training graphs: {len(train_graphs)}")
print(f"Number of validation graphs: {len(val_graphs)}")
print(f"Number of test graphs: {len(test_graphs)}")

In [ ]:
"""Графовая сверточная нейросеть"""
# Нормализация 
# Проверяю наличие пустых графов

empty_graphs = [i for i, graph in enumerate(graph_data_list) if graph.x.shape[0] == 0]
print(f"Empty graphs indices: {empty_graphs}")

In [ ]:
"""Графовая сверточная нейросеть"""
# Нормализация 
# Извлекаю все признаки тренировочных графов для обучения StandardScaler()

node_features = np.vstack([graph.x.numpy() for graph in train_graphs])
target_values = np.vstack([graph.y.numpy() for graph in train_graphs])

feature_scaler = StandardScaler().fit(node_features)
target_scaler = StandardScaler().fit(target_values)

In [ ]:
"""Графовая сверточная нейросеть"""
# Нормализация 
# Функция преобразует признаки в тренировочном, валидационном и тестовом сетах 

def transform_graphs(graphs, feature_scaler, target_scaler):
    for graph in graphs:
        graph.x = torch.tensor(feature_scaler.transform(graph.x.numpy()), dtype=torch.float)
        graph.y = torch.tensor(target_scaler.transform(graph.y.numpy().reshape(1, -1)), dtype=torch.float).view(-1)

transform_graphs(train_graphs, feature_scaler, target_scaler)
transform_graphs(val_graphs, feature_scaler, target_scaler)
transform_graphs(test_graphs, feature_scaler, target_scaler)

In [ ]:
"""Графовая сверточная нейросеть"""
# Архитерктура
# Параметры "Early Stopping"

num_epochs = 400
patience = 10
min_delta = 0.0001
best_val_loss = float('inf')
epochs_no_improve = 0

In [ ]:
"""Графовая сверточная нейросеть"""
# Архитерктура
# Поиск оптимальных гиперпараметров с помощью пятикратной "KFold" кросс-валидации
# Создаю класс ГСН, который принимает гиперпараметры (такой же класс будет создан еще раз, но уже для обучения и оценки)

param_grid = {
    'hidden_dim': [16, 32, 64],
    'num_layers': [4, 8, 12],
    'learning_rate': [0.00001, 0.0001, 0.001],
    'optimizer': ['Adam', 'SGD', 'RMSprop', 'Adagrad'],
    'activation_function': ['relu', 'sigmoid', 'tanh', 'softmax']
}

results = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, activation_function):
        super(GCN, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.activation_function = activation_function

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs:
            x = conv(x, edge_index)
            x = getattr(F, self.activation_function)(x)
        x = torch.mean(x, dim=0)
        x = self.fc(x)
        return x

for train_index, val_index in kf.split(train_val_graphs):
    train_graphs = [train_val_graphs[i] for i in train_index]
    val_graphs = [train_val_graphs[i] for i in val_index]

    for params in ParameterGrid(param_grid):
        hidden_dim = params['hidden_dim']
        num_layers = params['num_layers']
        learning_rate = params['learning_rate']
        optimizer_name = params['optimizer']
        activation_function = params['activation_function']
        
        print(f"Training with Hyperparameters: Hidden Dim={hidden_dim}, Num Layers={num_layers}, Learning Rate={learning_rate}, Optimizer={optimizer_name}, Activation Function={activation_function}")
        
        input_dim = node_features.shape[1]
        output_dim = 7
        
        model = GCN(input_dim, hidden_dim, output_dim, num_layers, activation_function)
        criterion = nn.MSELoss()
        
        optimizer = None
        if optimizer_name == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        elif optimizer_name == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        elif optimizer_name == 'RMSprop':
            optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
        elif optimizer_name == 'Adagrad':
            optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {optimizer_name}")
        
        for epoch in range(num_epochs):
            model.train()
            for data in train_graphs:
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, data.y)
                loss.backward()
                optimizer.step()
        
        model.eval()
        val_losses = []
        for data in val_graphs:
            with torch.no_grad():
                output = model(data)
                val_loss = mean_squared_error(data.y.numpy(), output.numpy())
                val_losses.append(val_loss)
        
        avg_val_loss = np.mean(val_losses)
        print(f"Avg Validation Loss: {avg_val_loss:.4f}")
        
        results.append({
            'Hidden Dim': hidden_dim,
            'Num Layers': num_layers,
            'Learning Rate': learning_rate,
            'Optimizer': optimizer_name,
            'Activation Function': activation_function,
            'Validation Loss': avg_val_loss
        })

best_params = min(results, key=lambda x: x['Validation Loss'])
print("Best Hyperparameters:", best_params)

In [ ]:
"""Графовая сверточная нейросеть"""
# Наилучшая комбинация гиперпараметров

best_hidden_dim = best_params['Hidden Dim']
best_num_layers = best_params['Num Layers']
best_learning_rate = best_params['Learning Rate']
best_optimizer_name = best_params['Optimizer']
best_activation_function = best_params['Activation Function']

In [ ]:
"""Графовая сверточная нейросеть"""
# Сохраняю тип наилучшего оптимайзера для последующей ссылки на него

best_optimizer = None
if best_optimizer_name == 'Adam':
    best_optimizer = torch.optim.Adam(model.parameters(), lr=best_learning_rate)
elif best_optimizer_name == 'SGD':
    best_optimizer = torch.optim.SGD(model.parameters(), lr=best_learning_rate)
elif best_optimizer_name == 'RMSprop':
    best_optimizer = torch.optim.RMSprop(model.parameters(), lr=best_learning_rate)
elif best_optimizer_name == 'Adagrad':
    best_optimizer = torch.optim.Adagrad(model.parameters(), lr=best_learning_rate)

In [ ]:
"""Графовая сверточная нейросеть"""
# Создал модель ГСН, которая принимает лучшие значения гиперпараметров после кросс-валидации
    
best_model = GCN(input_dim, best_hidden_dim, output_dim, best_num_layers, best_activation_function, best_optimizer)

In [ ]:
"""Графовая сверточная нейросеть"""
# Обучение и валидация модели ГСН
# Применил "Early Stopping" для исключения возможности переобучения
# Веса лучшей эпохи сохранил 
# Построил Train & Validation кривые

best_model_weights = None
best_val_loss = float('inf')
epochs_no_improve = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    total_loss = 0
    for data in train_val_graphs:
        best_model.train()
        best_optimizer.zero_grad()
        output = best_model(data)
        loss = criterion(output, data.y)
        loss.backward()
        best_optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_val_graphs)
    train_losses.append(avg_train_loss)
    
    val_epoch_losses = []
    for data in val_graphs:
        best_model.eval()
        with torch.no_grad():
            output = best_model(data)
            val_loss = mean_squared_error(data.y.numpy(), output.numpy())
            val_epoch_losses.append(val_loss)
    avg_val_loss = np.mean(val_epoch_losses)
    val_losses.append(avg_val_loss)
    
    print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
    if avg_val_loss < best_val_loss - min_delta:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        best_model_weights = best_model.state_dict()
    else:
        epochs_no_improve += 1
    
    if epochs_no_improve == patience:
        print(f'Early stopping after {epoch + 1} epochs.')
        break

if best_model_weights is not None:
    best_model.load_state_dict(best_model_weights)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

In [ ]:
"""Графовая сверточная нейросеть"""
# Оцениваю модель на тестовой выборке
# Применяю inverse_tranform()
# Создаю массивы с предсказанными и фактическими значениями для последующей визуализации

test_losses = []
test_predictions = []
original_targets = []

for data in test_graphs:
    best_model.eval()
    with torch.no_grad():
        output = best_model(data)
        test_loss = criterion(output, data.y).item()
        test_losses.append(test_loss)
        
        model_output_unscaled = target_scaler.inverse_transform(output.numpy().reshape(1, -1)).flatten()
        test_predictions.append(model_output_unscaled)
        
        original_target_unscaled = target_scaler.inverse_transform(data.y.numpy().reshape(1, -1)).flatten()
        original_targets.append(original_target_unscaled)

average_test_loss = np.mean(test_losses)
print(f"Average Test Loss of Scaled Data: {average_test_loss}")

mae = mean_absolute_error(np.concatenate(original_targets), np.concatenate(test_predictions))
print(f"Average Mean Absolute Error (MAE) of Inversed Data: {mae}")

original_targets = np.array(original_targets)
test_predictions = np.array(test_predictions)

In [ ]:
"""Графовая сверточная нейросеть"""
# Визуализация результатов

maes = []
num_targets = original_targets.shape[1]
target_names = [
    'Gibbs Free Energy (DFT)', 'Electronic Energy (DFT)', 'Entropy (DFT)', 
    'Enthalpy (DFT)', 'Dipole Moment (DFT)', 'HOMO (DFT)', 'LUMO (DFT)'
]

for i in range(num_targets):
    mae = mean_absolute_error(original_targets[:, i], test_predictions[:, i])
    maes.append(mae)

num_rows = 2
num_cols = (num_targets + 1) // num_rows
num_plots = num_rows * num_cols 

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18, 12))

if num_plots > 1:
    axes = axes.flatten()
else:
    axes = [axes] 

for i in range(num_targets):
    if i < num_plots: 
        ax = axes[i]
        ax.scatter(original_targets[:, i], test_predictions[:, i], alpha=0.5)
        ax.plot([original_targets[:, i].min(), original_targets[:, i].max()], 
                [original_targets[:, i].min(), original_targets[:, i].max()], 'r--')
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        ax.set_title(target_names[i])
        ax.grid(True)

        ax.text(0.05, 0.95, f"MAE: {maes[i]:.4f}", transform=ax.transAxes, fontsize=12,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

for j in range(num_targets, num_plots):
    axes[j].axis('off')

plt.tight_layout()
plt.show()